Steps adapted from [this tutorial](https://www.indiehackers.com/post/how-to-fine-tune-a-gpt-3-model-using-python-with-your-own-data-for-improved-performance-198dfe51d6).

#### Load Modules and Validate Data

- Data has already been created/set up via [gpt3_json_setup.py](gpt3_json_setup.py)
- If you have not created/retrieved a secret key from the OpenAI API, do so through [beta.openai.com](https://beta.openai.com)

In [2]:
import openai
from openai_credentials import skey

openai.api_key = skey
seattle_train = "./json_files/seattle_train.jsonl"
chicago_train = "./json_files/seattle_train.jsonl"
# validate data with $ openai tools fine_tunes.prepare_data -f CITY_train.jsonl

seattle_train_id = False
chicago_train_id = False

In [ ]:
seattle_upload_response = openai.File.create(
  file = open(seattle_train, "rb"),
  purpose = 'fine-tune'
)

seattle_train_id = seattle_upload_response.id
seattle_upload_response

In [ ]:
chicago_upload_response = openai.File.create(
  file = open(chicago_train, "rb"),
  purpose = 'fine-tune'
)

chicago_train_id = chicago_upload_response.id
chicago_upload_response

#### Begin Fine-Tuning

In [ ]:
if seattle_train_id:
    seattle_fine_tune_response = openai.FineTune.create(training_file=seattle_train_id)
    seattle_fine_tune_response

In [ ]:
if chicago_train_id:
    chicago_fine_tune_response = openai.FineTune.create(training_file=chicago_train_id)
    chicago_fine_tune_response

#### Check Fine-Tuning Progress

Option 1: use openai.FineTune.list_events() and pass in the CITY_fine_tune_response id to list all the current events. This will give you a list of all the events related to the fine-tuning job you're running:

In [ ]:
if seattle_train_id:
    seattle_fine_tune_events = openai.FineTune.list_events(id=seattle_fine_tune_response.id)
    seattle_fine_tune_events

if chicago_train_id:
    chicago_fine_tune_events = openai.FineTune.list_events(id=chicago_fine_tune_response.id)
    chicago_fine_tune_events

Option 2: use openai.FineTune.retrieve(id=CITY_fine_tune_response.id) to retrieve a specific fine-tune. This will give you a JSON object with information about a specific fine-tune job:

In [ ]:
if seattle_train_id:
    seattle_retrieve_response = openai.FineTune.retrieve(id=seattle_fine_tune_response.id)
    seattle_retrieve_response

if chicago_train_id:
    chicago_retrieve_response = openai.FineTune.retrieve(id=chicago_fine_tune_response.id)
    chicago_retrieve_response

#### Save Fine-Tuned Model

##### Troubleshooting fine_tuned_model as null

During the fine-tuning process, the fine_tuned_model key may not be immediately available in the fine_tune_response object returned by openai.FineTune.create().

To check the status of your fine-tuning process, you can call the openai.FineTune.retrieve() function and pass in the fine_tune_response.id. This function will return a JSON object with information about the training status, such as the current epoch, the current batch, the training loss, and the validation loss.

After the fine-tuning process is complete, you can check the status of all your fine-tuned models by calling openai.FineTune.list(). This will list all of your fine-tunes and their current status.

Once the fine-tuning process is complete, you can retrieve the fine_tuned_model key by calling the openai.FineTune.retrieve() function again and passing in the fine_tune_response.id. This will return a JSON object with the key fine_tuned_model and the ID of the fine-tuned model that you can use for further completions.

In [3]:
openai.FineTune.list()

<OpenAIObject list at 0x161e137aea0> JSON: {
  "data": [],
  "object": "list"
}

If CITY_fine_tuned_model != null

In [ ]:
if seattle_train_id:
    seattle_fine_tuned_model = seattle_fine_tune_response.fine_tuned_model
    seattle_fine_tuned_model

if chicago_train_id:
    chicago_fine_tuned_model = chicago_fine_tune_response.fine_tuned_model
    chicago_fine_tuned_model

If CITY_fine_tuned_model == null

In [4]:
if seattle_train_id:
    seattle_retrieve_response = openai.FineTune.retrieve(seattle_fine_tune_response.id)
    seattle_fine_tuned_model = seattle_retrieve_response.fine_tuned_model
    seattle_fine_tuned_model

if chicago_train_id:
    chicago_retrieve_response = openai.FineTune.retrieve(chicago_fine_tune_response.id)
    chicago_fine_tuned_model = chicago_retrieve_response.fine_tuned_model
    chicago_fine_tuned_model

#### Test Fine-Tuned Model

In [1]:
import jsonlines

seattle_prompts = []
with jsonlines.open("./json_files/seattle_test.jsonl") as dicts:
    seattle_prompts = [d["prompt"] for d in dicts]  # get all prompts from test set

chicago_prompts = []
with jsonlines.open("./json_files/seattle_test.jsonl") as dicts:
    chicago_prompts = [d["prompt"] for d in dicts]  # get all prompts from test set

# seattle_prompts
# chicago_prompts

In [5]:
answer = openai.Completion.create(
  model=fine_tuned_model,
  prompt=new_prompt,
  max_tokens=100,
  temperature=0
)
answer['choices'][0]['text']

NameError: name 'fine_tuned_model' is not defined